# MultiModal Document RAG with ColQwen2 and Llama 3.2 90B Vision
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/MultiModal_RAG_with_Nvidia_Investor_Slide_Deck.ipynb)

## Hardware Requirements
*To ensure the notebook runs faster please change the runtime type to T4 GPU:
`Runtime` -> `Change runtime type` -> `T4 GPU`*

*You can also run this notebook on a 16Gb M1 Macbook Pro

## Introduction

In this notebook we will see how to use Multimodal RAG to chat with Nvidia's invester slide deck from last year. The [slide deck](https://s201.q4cdn.com/141608511/files/doc_presentations/2023/Oct/01/ndr_presentation_oct_2023_final.pdf) is 39 pages with a combination of text, visuals, tables, charts and annotations. The document structure and templates vary from page to page and is quite difficult to RAG over using traditional methods.

We will be using a new multimodal approach!

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/Nvidia_collage.png?raw=1" width="500">

## MultiModal RAG Workflow

[ColPali](https://arxiv.org/abs/2407.01449) is a new multimodal retrieval system that seamlessly enables image retrieval.

By directly encoding image patches, it eliminates the need for optical character recognition (OCR), or image captioning to extract text from PDFs.

We will use `byaldi`, a library from [AnswerAI](https://www.answer.ai/), that makes it easier to work with an upgraded version of ColPali, called ColQwen2, to embed and retrieve images of our PDF documents.

Retrieved pages will then be passed into the Llama-3.2 90B Vision model served via a [Together AI](https://www.together.ai/) inference endpoint for it to answer questions.

To get a better explanation of how ColPali and the new Llama 3.2 Vision models work checkout the [blog post](https://www.together.ai/blog/multimodal-document-rag-with-llama-3-2-vision-and-colqwen2) connected to this notebook.

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/mmrag_only.png?raw=1" width="600">

### Install relevant libraries

In [1]:
!pip install byaldi together pdf2image

In [2]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (182 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 123632 

In [4]:
# Paste in your Together AI API Key or load it
api_key =  "efd2c374a3dd5e77756d162ba222ce8ce5f4d3da58056fc5762e5aca074918c9"

### Initialize the ColPali Model

In [29]:
import os
from pathlib import Path
from byaldi import RAGMultiModalModel

# Initialize RAGMultiModalModel
model = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v0.1")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### The document we will be retrieving from is a 39 page Nvidia investor presentation from 2023: [Investor Presentation October 2023](https://s201.q4cdn.com/141608511/files/doc_presentations/2023/Oct/01/ndr_presentation_oct_2023_final.pdf)

In [8]:
# Dowload and rename the last presentation from Nvidia to investors
!wget https://bigtruckrental.com/wp-content/uploads/2019/01/Zero-Radius.pdf


--2024-12-03 18:37:36--  https://bigtruckrental.com/wp-content/uploads/2019/01/Zero-Radius.pdf
Resolving bigtruckrental.com (bigtruckrental.com)... 141.193.213.10, 141.193.213.11
Connecting to bigtruckrental.com (bigtruckrental.com)|141.193.213.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10505023 (10M) [application/pdf]
Saving to: ‘Zero-Radius.pdf’

Zero-Radius.pdf     100%[===================>]  10.02M  18.2MB/s    in 0.6s    

2024-12-03 18:37:37 (18.2 MB/s) - ‘Zero-Radius.pdf’ saved [10505023/10505023]



### Lets create our index that will store the embeddings for the page images.

Caution: This cell below takes ~5 mins to index the whole PDF!

In [9]:
# Use ColQwen2 to index and store the presentation
index_name = "nvidia_index"
model.index(input_path=Path("/content/Zero-Radius.pdf"),
    index_name=index_name,
    store_collection_with_index=True, # Stores base64 images along with the vectors
    overwrite=True
)

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of document 0 to index.
Added page 26 of document 0 to index.
Added page 27 of docu

{0: '/content/Zero-Radius.pdf'}

### This concludes the indexing of the PDF phase - everything below happens at query time.

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/colpali_arch.png?raw=1" width="700">

### Let's query our indexed document.

Here the important thing to note is that the query is asking for details that are found on page 25 of the PDF!

In [ ]:
# Lets query our index and retrieve the page that has content with the highest similarity to the query

# The Data Centre revenue results are on page 25 - for context!
query = "What are the half year data centre renevue results and the 5 year CAGR for Nvidia data centre revenue?"
results = model.search(query, k=5)

print(f"Search results for '{query}':")
for result in results:
    print(f"Doc ID: {result.doc_id}, Page: {result.page_num}, Score: {result.score}")

print("Test completed successfully!")

Search results for 'What are the half year data centre renevue results and the 5 year CAGR for Nvidia data centre revenue?':
Doc ID: 0, Page: 25, Score: 25.875
Doc ID: 0, Page: 24, Score: 25.0
Doc ID: 0, Page: 28, Score: 23.75
Doc ID: 0, Page: 32, Score: 23.75
Doc ID: 0, Page: 31, Score: 23.75
Test completed successfully!


### Notice that ColQwen2 is able to retrieve that correct page with the highest similarity!

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/page_25.png?raw=1" width="700">

### How does this work? What happens under the hood between the different pages and query token?

The interaction operation between page image patch and query text token representations to score each page of the document is what allows this great retreival performance.

Typically each image is resized and cut into patch sizes of 16x16 pixels. These patches are then embedded into 128 dimensional vectors which are stored and used to perform the MaxSim and late interaction operations between the image and text tokens. ColPali is a multi-vector approach because it produces multiple vectors for each image/query; one vector for each token instead of just one vector for all tokens.

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/ColPaliMaxSim-1.png?raw=1" width="700">

The retrieval step takes about 185 ms.

In [ ]:
%%timeit
model.search(query, k=5)

182 ms ± 4.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Lets now pass in the retrieved page to the Llama-3.2 90B Vision Model.

This model will read the question: `"What are the half year data centre renevue results and the 5 year CAGR for Nivida data centre revenue?"`

And take in the retrieved page and produce an answer!

You can pass in a URL to the image of the retrieved page or a base64 encoded version of the image.

In [42]:
# Since we stored the collection along with the index we have the base64 images of all PDF pages aswell!
query=" list all Refuse Vehicle Button Functions"
model.search(query, k=1)

[{'doc_id': 0, 'page_num': 41, 'score': 14.75, 'metadata': {}, 'base64': '...'}]

In [46]:
#query = "where is the clean out door located?"
query="list all Refuse Vehicle Button Functions"
#query = "tell me about Refuse Vehicle Button Functions"
returned_page = model.search(query, k=3)[0].base64
#returned_page = model.search(query, k=3)
#returned_pages = [
#          {"type": "image_url", "image_url": {  "url": f"data:image/jpeg;base64,{page.base64}"}}
#          for page in returned_page
#        ]

## We'll use a [Together AI](together.ai) inference endpoint to access the Llama-3.2 90B Vision model

In [47]:
import os
from together import Together

client = Together(api_key = api_key)

response = client.chat.completions.create(
  model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": query}, #query
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{returned_page}", #retrieved page image
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0].message.content)


The Refuse Vehicle Button Functions are:

1. PUMP Button: Press to activate Hydraulic functions. Press again to disable Hydraulic functions.
2. TAILGATE OPEN Button: Press and hold the button to open the tailgate. You must also press and hold the ACTIVATE button while holding the TAILGATE OPEN button.
3. ACTIVATE Button: Press and hold the ACTIVATE button to enable the Tailgate functions or the Arm functions.
4. REMOTE Button: Press the REMOTE button to enable the remote control button panel. Press the button again to disable the remote control button panel.
5. TAG AXLE RAISE Button (OPTIONAL): Press the button to raise the tag axle.
6. ARM ON Button: Press and hold the enable the Arm functions. You must also press and hold the ACTIVATE button while holding the ARM ON button.

These buttons are used to operate various functions on a refuse vehicle, including hydraulic functions, tailgate operation, remote control, and arm functions.


Here we can see that the combination of ColQwen2 as a image retriever and Llama-3.2 90B Vision is a powerful duo for multimodal RAG applications specially with PDFs.

Not only was ColQwen2 able to retrieve the correct page that had the right answer on it but then Llama-3.2 90B Vision was also able to find exactly where on the page this answer was, ignoring all the irrelvant details!

Voila!🎉🎉

Learn more about Llama 3.2 Vision in the [docs](https://docs.together.ai/docs/vision-overview) here!